In [ ]:
from openai import AzureOpenAI
import os
# print(os.getcwd())
os.chdir("..")
# print(os.getcwd())

from config import Config

envConfig = Config()

client = AzureOpenAI(
    api_key=envConfig.openai_api_key,
    api_version=envConfig.openai_api_version,
    azure_endpoint=envConfig.openai_api_base_url
)

In [ ]:
from langchain_openai.chat_models import AzureChatOpenAI

model = AzureChatOpenAI(
    api_key=envConfig.openai_api_key,
    api_version=envConfig.openai_api_version,
    azure_endpoint=envConfig.openai_api_base_url,
    azure_deployment="gpt-turbo",
    temperature=.9

)

prompt = "I am script writter for Movies, please suggest a nice name for a Action movie."
model.invoke(prompt)

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke(prompt)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_movie_name = PromptTemplate(
    input_variables=['Genre'],
    template="I am script writter for Movies, please suggest a nice name for a {Genre} movie."
)

prompt_movie_name.format(Genre="Action")

chain_movie_name = LLMChain(llm =  model, prompt = prompt_movie_name)

movie_name = chain_movie_name.run({
    "Genre":"Comedy"
})

print(movie_name)

In [ ]:
prompt_movie_story = PromptTemplate(
    input_variables=['movie_name'],
    template="I want to write the story of the movie {movie_name}, please give a story line in about 100 words."
)

chain_movie_story = LLMChain(llm =  model, prompt = prompt_movie_story)

chain_movie_story.run({
    "movie_name":movie_name
})

Simple Sequential Chain

In [ ]:

from langchain.chains import SimpleSequentialChain

# Create a SimpleSequentialChain with the defined prompts and chain
simple_seq_chain = SimpleSequentialChain(chains=[chain_movie_name, chain_movie_story])

simple_seq_chain.run("Suspense")


Sequential Chain ( Stateful)

In [ ]:
from langchain.chains import SequentialChain

prompt_movie_name = PromptTemplate(
    input_variables=['Genre'],
    template="I am script writter for Movies, please suggest a nice name for a {Genre} movie."
)

chain_movie_name = LLMChain(llm =  model, prompt = prompt_movie_name, output_key="movie_name")


prompt_movie_story = PromptTemplate(
    input_variables=['movie_name','Genre'],
    template="I want to write the story of the movie {movie_name} of Genre {Genre}, please give a story line in about 100 words."
)

chain_movie_story = LLMChain(llm =  model, prompt = prompt_movie_story, output_key="movie_story")

seq_chain = SequentialChain(
    chains=[chain_movie_name, chain_movie_story],
    input_variables=['Genre'],
    output_variables=["movie_name", "movie_story"]
)

seq_chain.invoke("Thriller")

Embeddings

In [ ]:
from langchain_openai.embeddings import AzureOpenAIEmbeddings

embedding_model = AzureOpenAIEmbeddings(
    api_key=envConfig.openai_api_key,
    api_version=envConfig.openai_api_version,
    azure_endpoint=envConfig.openai_api_base_url,
    azure_deployment="embedding-model"
)

vector = embedding_model.embed_query("I like Avengers!!")
print(vector[:5])

Training Dataset

In [ ]:
movie1= embedding_model.embed_query("After the mysterious disappearance of her daughter, a grieving mother turns to a psychic for help. As the psychic begins to uncover the truth behind the girl's disappearance, it becomes clear that there is something sinister happening in the small town. With the help of the local sheriff, they set out to uncover the dark secrets that have been hidden for years, but as they get closer to the truth, they realize that there are forces at work that are far beyond anything they could have imagined. Will they be able to solve the mystery and bring the girl home, or will they be consumed by the shadowed secrets that lie buried in the town's troubled past?")
movie2= embedding_model.embed_query("Laugh Out Loud follows the story of Lola, a high school student who must navigate the ups and downs of teenage life while also trying to mend her strained relationship with her separated parents. When her mom sends her to a new school, she discovers the world of stand-up comedy and decides to pursue a career in it. Along the way, Lola learns about the power of forgiveness and the importance of following your dreams")
movie3=embedding_model.embed_query("Deadly Pursuit is a gripping thriller that follows a young woman named Melanie Fraser, who is on the run from a dangerous drug dealer after witnessing a murder. She seeks help from a former soldier, Shane Alcott, who is haunted by his past and reluctant to get involved. As they flee through the wilderness, they are pursued by the ruthless killer and his henchmen. With danger at every turn, Melanie and Shane must trust each other to stay alive and outsmart their pursuers before it's too late.")


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

response = seq_chain.invoke("Action")

story = response.get("movie_story")

print(story)

current_movie_query_vector = embedding_model.embed_query(story)

similarity = cosine_similarity([movie1],[current_movie_query_vector])

similarity[0][0]